In [2]:
import implicit
import pandas as pd
import scipy
import annoy
import nmslib
import numpy

In [20]:
import pprint

In [3]:
df = pd.read_csv("final_data.csv", index_col=0)
df.drop("index",axis=1,inplace=True)
df.head()

,movieId,title,genres,imdbId,tmdbId,"('count', 0.5)","('count', 1.0)","('count', 1.5)","('count', 2.0)","('count', 2.5)",...,"('count', 3.5)","('count', 4.0)","('count', 4.5)","('count', 5.0)",total_count,percent_0.5to1,percent_4.5to5,percent_polarity,total_tails,Cluster
0,56,Kids of the Round Table (1995),Adventure|Children|Comedy|Fantasy,tt0113541,124057.0,4.0,33.0,2.0,15.0,2.0,...,2.0,12.0,0.0,2.0,83.0,0.445783,0.024096,0.421687,0.469880,0
1,66,Lawnmower Man 2: Beyond Cyberspace (1996),Action|Sci-Fi|Thriller,tt0116839,11525.0,135.0,796.0,65.0,500.0,62.0,...,26.0,172.0,2.0,100.0,2586.0,0.360015,0.039443,0.320572,0.399459,0
2,181,Mighty Morphin Power Rangers: The Movie (1995),Action|Children,tt0113820,9070.0,310.0,970.0,99.0,534.0,130.0,...,81.0,159.0,24.0,92.0,3001.0,0.426524,0.038654,0.387871,0.465178,0
3,193,Showgirls (1995),Drama,tt0114436,10802.0,671.0,2095.0,465.0,1712.0,519.0,...,360.0,684.0,109.0,403.0,8937.0,0.309500,0.057290,0.252210,0.366790,0
4,285,Beyond Bedlam (1993),Drama|Horror,tt0106402,56428.0,4.0,27.0,3.0,14.0,0.0,...,0.0,7.0,0.0,10.0,88.0,0.352273,0.113636,0.238636,0.465909,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 482
Data columns (total 21 columns):
movieId             476 non-null int64
title               476 non-null object
genres              476 non-null object
imdbId              476 non-null object
tmdbId              473 non-null float64
('count', 0.5)      476 non-null float64
('count', 1.0)      476 non-null float64
('count', 1.5)      476 non-null float64
('count', 2.0)      476 non-null float64
('count', 2.5)      476 non-null float64
('count', 3.0)      476 non-null float64
('count', 3.5)      476 non-null float64
('count', 4.0)      476 non-null float64
('count', 4.5)      476 non-null float64
('count', 5.0)      476 non-null float64
total_count         476 non-null float64
percent_0.5to1      476 non-null float64
percent_4.5to5      476 non-null float64
percent_polarity    476 non-null float64
total_tails         476 non-null float64
Cluster             476 non-null int64
dtypes: float64(16), int64(2), object(3)


## Implicit

In [5]:
df_sparse = pd.read_csv("item_item_final.csv", index_col=0)
df_sparse.drop("timestamp",axis=1,inplace=True)
df_sparse.head()

,userId,movieId,rating
66,4,66,0.5
90,4,181,0.5
93,4,193,0.5
159,4,546,0.5
296,4,1760,0.5


In [6]:
sparse = scipy.sparse.csr_matrix(df_sparse.values)

In [7]:
als_model = implicit.als.AlternatingLeastSquares()

In [8]:
als_model.fit(sparse)

100%|██████████| 15.0/15 [00:39<00:00,  2.61s/it]


In [9]:
sim_items = als_model.similar_items(70)

In [10]:
sim_items

[(70, 0.03112511),
 (64398, 0.016634775),
 (107603, 0.016568653),
 (119561, 0.016266637),
 (70296, 0.015890596),
 (175915, 0.015756084),
 (45600, 0.01571552),
 (69174, 0.015582978),
 (178316, 0.015449081),
 (11806, 0.0154413525)]

In [12]:
sim_items[0][0]

70

In [46]:
df_sparse.iloc[[64398],[1]]

,movieId
8324808,1681


In [65]:
def get_recs(movie):
    """Takes a index from the model and makes a five recommendations."""
    id_list = []
    sim_items = als_model.similar_items(movie)
    for item in sim_items:
        iid = df_sparse.iloc[item[0],[1]]
        id_list.append(int(iid[0]))
    final = []
    for rec in id_list:
        final.append(df.loc[df['movieId']== rec, 'title'].iloc[0])
    return final

    
    
    

In [68]:
get_recs(70)

['Disaster Movie (2008)',
 'Mortal Kombat: Annihilation (1997)',
 'Spy Kids 3-D: Game Over (2003)',
 'Showgirls (1995)',
 'Superman IV: The Quest for Peace (1987)',
 'Chopping Mall (a.k.a. Killbots) (1986)',
 'Alone in the Dark (2005)',
 'Grease 2 (1982)',
 'Alvin and the Chipmunks: The Squeakquel (2009)',
 'Pokémon 3: The Movie (2001)']

str